# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

### Function

#### expand_calc_features

In [2]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_list = []

    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)

    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')

    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)

    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    return result_df


#### expand_calc_features_robust

In [3]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_data = []

    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']

        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue

        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)

    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')

    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")

    return result_df

#### transform_data

## Transform data v1.1

In [4]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_data_v1_1(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v1.1",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],
            "Data_selection": row['Data_selection'],  # Data selection
            "Application_date": row['application_date'],
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df


#### PSI Functions new

In [5]:
## Updated on 27-10-2025 - Modified for Training Period Baseline
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify

    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []

    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue

        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)

    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame,
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features (deciles with fallback) and categorical features (top 6 + others)
    based on the entire training period data.

    Parameters:
    -----------
    df : pd.DataFrame
        Full input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    train_period_df : pd.DataFrame
        Training period dataframe (June 2024 to March 2025)

    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    binning_info = {}

    # Create bins for numerical features with fallback strategy
    for feature in numerical_features:
        valid_data = train_period_df[feature].dropna()

        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue

        bins = None
        bin_count = None

        # Try 10 bins (deciles)
        try:
            test_bins = np.percentile(valid_data, np.arange(0, 101, 10))
            test_bins = np.unique(test_bins)
            if len(test_bins) >= 11:  # 11 edges = 10 bins
                bins = test_bins
                bin_count = 10
        except Exception as e:
            pass

        # If 10 bins not possible, try 5 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, np.arange(0, 101, 20))
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 6:  # 6 edges = 5 bins
                    bins = test_bins
                    bin_count = 5
            except Exception as e:
                pass

        # If 5 bins not possible, try 3 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, [0, 33.33, 66.67, 100])
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 4:  # 4 edges = 3 bins
                    bins = test_bins
                    bin_count = 3
            except Exception as e:
                pass

        # If still no bins possible, use equal distance bins of 5
        if bins is None:
            print(f"Warning: Feature '{feature}' has insufficient variance - cannot create standard bins")
            print(f"Feature '{feature}': Using equal distance bins of 5")

            min_val = valid_data.min()
            max_val = valid_data.max()

            # Create 5 equal distance bins
            bins = np.linspace(min_val, max_val, 6)  # 6 edges = 5 bins
            bins = np.unique(bins)
            bin_count = len(bins) - 1

            # If all values are the same, add slight buffer
            if bin_count == 1:
                bins = np.array([min_val - 0.1, min_val, min_val + 0.1])
                bin_count = 2
                print(f"Feature '{feature}': Constant value ({min_val}). Created 2 equal distance bins with buffer")

        # Add infinity edges to capture all values
        bins = bins.copy()
        bins[0] = -np.inf
        bins[-1] = np.inf

        print(f"Feature '{feature}': Created {bin_count} bins")

        # Create bin ranges dictionary
        bin_ranges = {}
        for i in range(len(bins)-1):
            bin_name = f"Bin_{i+1}"
            bin_ranges[bin_name] = {
                'min': bins[i],
                'max': bins[i+1],
                'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
            }

        binning_info[feature] = {
            'type': 'numerical',
            'bins': bins,
            'bin_ranges': bin_ranges,
            'bin_count': bin_count
        }

    # Create bins for categorical features (top 6 + others) using training period
    for feature in categorical_features:
        value_counts = train_period_df[feature].value_counts()
        unique_categories = value_counts.index.tolist()
        print(f"Unique categories: {unique_categories}")

        if len(unique_categories) <= 6:
            # Treat each category as a separate bin
            top_categories = unique_categories
        else:
            # Use top 6 categories only
            top_categories = value_counts.nlargest(6).index.tolist()

        print(f"Top categories for feature '{feature}': {top_categories}")

        binning_info[feature] = {
                'type': 'categorical',
                'top_categories': top_categories,
                'bin_ranges': {}  # No ranges for categorical
            }

    return binning_info


def apply_binning(df: pd.DataFrame,
                  feature: str,
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature

    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)

        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]

        binned = pd.cut(df[feature],
                       bins=bins,
                       labels=labels,
                       include_lowest=True,
                       duplicates='drop')

        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'

        return binned

    else:  # categorical
        top_cats = binning_info['top_categories']

        # Convert to string for consistent comparison
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)

        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')

        # Convert top_cats to strings for comparison
        top_cats_str = [str(cat) for cat in top_cats]

        # Apply binning logic: use category name if in top_cats, else 'Others' (except for Missing)
        binned = feature_data.apply(lambda x: x if x in top_cats_str else ('Others' if x != 'Missing' else 'Missing'))

        return binned


def calculate_psi(expected_pct: pd.Series,
                  actual_pct: pd.Series,
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index with proper epsilon handling and renormalization.

    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)

    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)

    # Only add epsilon where values are zero
    expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
    actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)

    # Renormalize to ensure they sum to 1 after adding epsilon
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()

    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))

    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period (June 2024 to March 2025)
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column (identifies train period)
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []

    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(f"Feature binned {df[f'{feature}_binned']}")
        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate PSI for each test month
        for month in test_months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(train_baseline, actual_dist)

            # Calculate average percentages across all bins
            expected_avg_pct = train_baseline.mean() * 100
            actual_avg_pct = actual_dist.mean() * 100

            # # Count distinct accounts for segment
            # base_segment_count = train_segment[account_id_col].nunique()
            # actual_segment_count = actual_segment[account_id_col].nunique()


            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                'Current_Month': month,
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value
            })

    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(train_baseline, actual_dist)

                    # Calculate average percentages across all bins
                    expected_avg_pct = train_baseline.mean() * 100
                    actual_avg_pct = actual_dist.mean() * 100

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                        'Current_Month': month,
                        'Base_Count': base_segment_count,
                        'Actual_Count': actual_segment_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value
                    })

    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month',
                            data_selection_col: str = 'Data_selection',
                            account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature comparing training period
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []
    epsilon = 0.0001

    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(df[f'{feature}_binned'])

        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate bin-level PSI for each test month
        for month in test_months:
            month_data = df[df[month_col] == month]
            actual_dist = month_data[f'{feature}_binned'].value_counts(normalize=True)

            # Count distinct accounts
            base_count = df[df[data_selection_col] == 'Train'][account_id_col].nunique()
            actual_count = month_data[account_id_col].nunique()

            # Get all bins
            all_bins = train_baseline.index.union(actual_dist.index)

            for bin_name in all_bins:
                # Simplified epsilon logic - no redundancy
                expected_pct = train_baseline.get(bin_name, 0)
                actual_pct = actual_dist.get(bin_name, 0)

                # Add epsilon only if zero
                expected_pct = epsilon if expected_pct == 0 else expected_pct
                actual_pct = epsilon if actual_pct == 0 else actual_pct

                # Calculate bin-level PSI
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name

                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                    'Current_Month': month,
                    'Base_Count': base_count,
                    'Actual_Count': actual_count,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })

    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate bin-level PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    # Get all bins
                    all_bins = train_baseline.index.union(actual_dist.index)

                    for bin_name in all_bins:
                        # Simplified epsilon logic - no redundancy
                        expected_pct = train_baseline.get(bin_name, 0)
                        actual_pct = actual_dist.get(bin_name, 0)

                        # Add epsilon only if zero
                        expected_pct = epsilon if expected_pct == 0 else expected_pct
                        actual_pct = epsilon if actual_pct == 0 else actual_pct

                        # Calculate bin-level PSI
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name

                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                            'Current_Month': month,
                            'Base_Count': base_segment_count,
                            'Actual_Count': actual_segment_count,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })

    return pd.DataFrame(results)

# apps_score_cash

## Trench 1

In [9]:
sq = """ 
WITH
  base AS (
    SELECT
      r.customerId,
      r.digitalLoanAccountId,
      r.application_submission_date,
      CASE
        WHEN r.trenchCategory LIKE 'T1-A' THEN 'Trench 1'
        WHEN r.trenchCategory LIKE 'T2-A' THEN 'Trench 2'
        WHEN r.trenchCategory LIKE 'T3-A' THEN 'Trench 3'
        END trenchCategory,
      r.apps_score,
      r.cb_score,
      r.dl_score,
      coalesce(
        r.deviceOs,
        CASE
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%andro%'
            THEN 'android'
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%os%'
            THEN 'ios'
          WHEN lower(lmt.deviceType) LIKE '%andro%' THEN 'android'
          ELSE 'ios'
          END)
        osType,
      date(
        IF(
          lmt.new_loan_type = 'Flex-up',
          lmt.startApplyDateTime,
          lmt.termsAndConditionsSubmitDateTime))
        application_date,
      CASE
        WHEN dataset LIKE 'Dev_Train' THEN 'Dev_Train'
        WHEN dataset LIKE 'Dev_Test' THEN 'Dev_Test'
        WHEN dataset LIKE 'OOS' THEN 'Out_Of_Scope'
        END Data_selection
    FROM risk_mart.quick_applied_aug2024_nov2025_app_scored_extended r
    LEFT JOIN `risk_credit_mis.loan_master_table` lmt
      ON lmt.digitalLoanAccountId = r.digitalLoanAccountId
  )
SELECT
  customerId customer_id,
  digitalLoanAccountId,
  application_submission_date,
  trenchCategory,
  apps_score,
  cb_score,
  dl_score,
  osType,
  application_date,
  Data_selection
FROM base
WHERE
  trenchCategory = 'Trench 1'
  AND apps_score IS NOT NULL;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 0db43c29-fdb9-480c-92ed-74fa23ed8276 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (310159, 10)


In [10]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'application_submission_date',
       'trenchCategory', 'apps_score', 'cb_score', 'dl_score', 'osType',
       'application_date', 'Data_selection'],
      dtype='object')

In [14]:
feature_column = ['apps_score', 'cb_score', 'dl_score']

dfd = transform_data_v1_1(data, feature_column, a='apps_score', modelDisplayName='apps_score_cash', tc='Trench 1', subscription_name = 'Cash December APPs Models Upgrade') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310159 entries, 0 to 310158
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            310159 non-null  int64  
 1   digitalLoanAccountId  310159 non-null  object 
 2   crifApplicationId     310159 non-null  object 
 3   prediction            310159 non-null  float64
 4   start_time            310159 non-null  object 
 5   end_time              310159 non-null  object 
 6   modelDisplayName      310159 non-null  object 
 7   modelVersionId        310159 non-null  object 
 8   calcFeature           310159 non-null  object 
 9   subscription_name     310159 non-null  object 
 10  message_id            310159 non-null  object 
 11  publish_time          310159 non-null  object 
 12  attributes            310159 non-null  object 
 13  trenchCategory        310159 non-null  object 
 14  deviceOs              310159 non-null  object 
 15  

In [15]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,30691,2025-07-01,2025-08-25
1,Dev_Train,219533,2024-08-13,2025-06-30
2,Out_Of_Scope,59935,2025-08-26,2026-01-06


In [16]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3621715,969c2b8f-5076-401c-91e4-06f88363c581,590c7e6f-4d65-4795-bf4e-110221623f98,0.531848,2026-01-27T16:51:52.191558,2026-01-27T16:51:52.191558,apps_score_cash,v1.1,"{""apps_score"": 0.5318482553475778, ""cb_score"":...",Cash December APPs Models Upgrade,b4b888d8-18cf-42ee-821f-d1f35be83bac,2026-01-27T16:51:52.191558,{},Trench 1,Android,Dev_Test,2025-08-20
1,3540009,a1d46ba5-9736-4361-b7de-5b24080670c8,3038dc86-f25c-44df-8af6-92c84e90db4c,0.473363,2026-01-27T16:51:52.191558,2026-01-27T16:51:52.191558,apps_score_cash,v1.1,"{""apps_score"": 0.4733634909950922, ""cb_score"":...",Cash December APPs Models Upgrade,1f0ef776-0a67-4b7c-a78a-a9563025816a,2026-01-27T16:51:52.191558,{},Trench 1,Android,Dev_Test,2025-07-09
2,3621221,f717e2f4-4d16-4985-a43d-b4364e88ecc9,b579f204-782e-43c2-904f-fcc297e01768,0.484682,2026-01-27T16:51:52.191558,2026-01-27T16:51:52.191558,apps_score_cash,v1.1,"{""apps_score"": 0.4846818876086752, ""cb_score"":...",Cash December APPs Models Upgrade,bf1f4c05-516e-44a7-aa70-168b8e7e364e,2026-01-27T16:51:52.191558,{},Trench 1,Android,Dev_Test,2025-08-20
3,3561094,c82e6da3-eb7a-417b-9e56-5d5125e233ea,fd245e10-1789-4bdc-974d-2bda05c15f25,0.491497,2026-01-27T16:51:52.191558,2026-01-27T16:51:52.191558,apps_score_cash,v1.1,"{""apps_score"": 0.49149691492433734, ""cb_score""...",Cash December APPs Models Upgrade,b4eb78ee-de8a-4d30-a21d-9c1e756311fe,2026-01-27T16:51:52.191558,{},Trench 1,Android,Dev_Test,2025-07-16
4,3562750,61657fb9-2e01-4dcc-bedc-171b731c4f98,92c04562-d6e5-4391-aa0f-02fb5e103b91,0.477767,2026-01-27T16:51:52.191558,2026-01-27T16:51:52.191558,apps_score_cash,v1.1,"{""apps_score"": 0.4777673283643413, ""cb_score"":...",Cash December APPs Models Upgrade,72a311fd-0221-4382-9d8d-318cd035930c,2026-01-27T16:51:52.191558,{},Trench 1,Android,Dev_Test,2025-07-17


In [17]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8de77296-fb9a-4329-bbd9-073ce165c80b>

## Trench 2

In [18]:
sq = """ 
WITH
  base AS (
    SELECT
      r.customerId,
      r.digitalLoanAccountId,
      r.application_submission_date,
      CASE
        WHEN r.trenchCategory LIKE 'T1-A' THEN 'Trench 1'
        WHEN r.trenchCategory LIKE 'T2-A' THEN 'Trench 2'
        WHEN r.trenchCategory LIKE 'T3-A' THEN 'Trench 3'
        END trenchCategory,
      r.apps_score,
      r.cb_score,
      r.dl_score,
      coalesce(
        r.deviceOs,
        CASE
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%andro%'
            THEN 'android'
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%os%'
            THEN 'ios'
          WHEN lower(lmt.deviceType) LIKE '%andro%' THEN 'android'
          ELSE 'ios'
          END)
        osType,
      date(
        IF(
          lmt.new_loan_type = 'Flex-up',
          lmt.startApplyDateTime,
          lmt.termsAndConditionsSubmitDateTime))
        application_date,
      CASE
        WHEN dataset LIKE 'Dev_Train' THEN 'Dev_Train'
        WHEN dataset LIKE 'Dev_Test' THEN 'Dev_Test'
        WHEN dataset LIKE 'OOS' THEN 'Out_Of_Scope'
        END Data_selection
    FROM risk_mart.quick_applied_aug2024_nov2025_app_scored_extended r
    LEFT JOIN `risk_credit_mis.loan_master_table` lmt
      ON lmt.digitalLoanAccountId = r.digitalLoanAccountId
  )
SELECT
  customerId customer_id,
  digitalLoanAccountId,
  application_submission_date,
  trenchCategory,
  apps_score,
  cb_score,
  dl_score,
  osType,
  application_date,
  Data_selection
FROM base
WHERE
  trenchCategory = 'Trench 2'
  AND apps_score IS NOT NULL;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 5f59361b-2d4b-45e6-8d8d-6ba0228f9c61 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (120001, 10)


In [19]:
feature_column = ['apps_score', 'cb_score', 'dl_score']

dfd = transform_data_v1_1(data, feature_column, a='apps_score', modelDisplayName='apps_score_cash', tc='Trench 2', subscription_name = 'Cash December APPs Models Upgrade') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120001 entries, 0 to 120000
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            120001 non-null  int64  
 1   digitalLoanAccountId  120001 non-null  object 
 2   crifApplicationId     120001 non-null  object 
 3   prediction            120001 non-null  float64
 4   start_time            120001 non-null  object 
 5   end_time              120001 non-null  object 
 6   modelDisplayName      120001 non-null  object 
 7   modelVersionId        120001 non-null  object 
 8   calcFeature           120001 non-null  object 
 9   subscription_name     120001 non-null  object 
 10  message_id            120001 non-null  object 
 11  publish_time          120001 non-null  object 
 12  attributes            120001 non-null  object 
 13  trenchCategory        120001 non-null  object 
 14  deviceOs              120001 non-null  object 
 15  

In [20]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,13484,2025-07-01,2025-08-25
1,Dev_Train,72351,2024-08-13,2025-06-30
2,Out_Of_Scope,34166,2025-08-26,2025-12-03


In [21]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,2283077,f49fb2bd-cc4f-4810-b1e3-4306d74486fb,e329dcbf-e8c4-4929-9b63-1c3a7d7487e6,0.510840,2026-01-27T16:59:38.987597,2026-01-27T16:59:38.987597,apps_score_cash,v1.1,"{""apps_score"": 0.510840095359558, ""cb_score"": ...",Cash December APPs Models Upgrade,17bb9b06-2a29-4120-a81e-d4a2059311ac,2026-01-27T16:59:38.987597,{},Trench 2,Android,Dev_Test,2025-08-06
1,3470731,6c911c9b-b075-498e-8b44-f80738ab192c,8efbed12-bdd8-4ad6-988f-8fc87c183507,0.504725,2026-01-27T16:59:38.987597,2026-01-27T16:59:38.987597,apps_score_cash,v1.1,"{""apps_score"": 0.5047251034338015, ""cb_score"":...",Cash December APPs Models Upgrade,dfee5992-6d35-4ef0-8ada-be40bb0da7f7,2026-01-27T16:59:38.987597,{},Trench 2,Android,Dev_Test,2025-08-07
2,2265742,e006c975-bcdc-4b94-9004-0bccce150379,3be14ad7-d61a-4f39-b280-da5113828abf,0.531777,2026-01-27T16:59:38.987597,2026-01-27T16:59:38.987597,apps_score_cash,v1.1,"{""apps_score"": 0.5317768933801108, ""cb_score"":...",Cash December APPs Models Upgrade,15da369c-28f1-4a26-a381-57910a4c67ba,2026-01-27T16:59:38.987597,{},Trench 2,Android,Dev_Test,2025-08-11
3,2806887,5f3a7564-3670-49c8-ba5f-f72025a0a674,4b8b952c-d39f-4329-ab87-afa9e744c5d9,0.519685,2026-01-27T16:59:38.987597,2026-01-27T16:59:38.987597,apps_score_cash,v1.1,"{""apps_score"": 0.5196854333940797, ""cb_score"":...",Cash December APPs Models Upgrade,59b124f5-6d48-4ecd-8251-08a8edf313d5,2026-01-27T16:59:38.987597,{},Trench 2,Android,Dev_Test,2025-08-12
4,3416954,dc039895-aa5b-41bc-9d81-ab02b0ec2674,e1343a1c-899f-4c66-83a2-60ecac2a119e,0.502044,2026-01-27T16:59:38.987597,2026-01-27T16:59:38.987597,apps_score_cash,v1.1,"{""apps_score"": 0.5020440099406779, ""cb_score"":...",Cash December APPs Models Upgrade,f4831916-39e9-4981-afe9-b32c8726321c,2026-01-27T16:59:38.987597,{},Trench 2,Android,Dev_Test,2025-08-13


In [22]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=75efa78d-3a0c-4e4d-80f1-a26ff9c0b537>

# Beta_stack_Model_cash

## Trench 2

In [24]:
sq = """ 
WITH
  base AS (
    SELECT
      r.customer_id,
      r.digitalLoanAccountId,
      r.ln_appln_submit_datetime,
      r.trench_category,
      r.demo_score,
      r.apps_score,
      r.credo_score,
      r.stack_score,
      coalesce(
        CASE
          WHEN lower(r.ln_os_type) LIKE '%andro%' THEN 'android'
          WHEN lower(r.ln_os_type) LIKE '%os%' THEN 'ios'
          WHEN r.ln_os_type IS NULL THEN 'ios'
          ELSE 'ios'
          END,
        CASE
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%andro%'
            THEN 'android'
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%os%'
            THEN 'ios'
          WHEN lower(lmt.deviceType) LIKE '%andro%' THEN 'android'
          ELSE 'ios'
          END)
        osType,
      date(
        IF(
          lmt.new_loan_type = 'Flex-up',
          lmt.startApplyDateTime,
          lmt.termsAndConditionsSubmitDateTime))
        application_date,
      CASE
        WHEN dev_split LIKE 'Dev_Train' THEN 'Dev_Train'
        WHEN dev_split LIKE 'Dev_Test' THEN 'Dev_Test'
        ELSE dev_split
        END Data_selection
    FROM
      `worktable_data_analysis.cash_beta_trench1_applied_loans_backscored_20241001_20251130`
        r
    LEFT JOIN `risk_credit_mis.loan_master_table` lmt
      ON lmt.digitalLoanAccountId = r.digitalLoanAccountId
    WHERE lmt.disbursementDateTime IS NOT NULL
  )
SELECT 
customer_id,
digitalLoanAccountId,
ln_appln_submit_datetime,
trench_category,
demo_score,
apps_score,
credo_score,
stack_score Beta_cash_stack_score,
osType,
application_date,
Data_selection,
FROM base
WHERE Data_selection IS NOT NULL
and osType = 'android';

"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID f9120ba4-105b-418f-911b-0d9e6aaff8e1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (9468, 11)


In [25]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'Beta_cash_stack_score']

dfd = transform_data_v1_1(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash December APPs Models Upgrade') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9468 entries, 0 to 9467
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            9468 non-null   int64  
 1   digitalLoanAccountId  9468 non-null   object 
 2   crifApplicationId     9468 non-null   object 
 3   prediction            9468 non-null   float64
 4   start_time            9468 non-null   object 
 5   end_time              9468 non-null   object 
 6   modelDisplayName      9468 non-null   object 
 7   modelVersionId        9468 non-null   object 
 8   calcFeature           9468 non-null   object 
 9   subscription_name     9468 non-null   object 
 10  message_id            9468 non-null   object 
 11  publish_time          9468 non-null   object 
 12  attributes            9468 non-null   object 
 13  trenchCategory        9468 non-null   object 
 14  deviceOs              9468 non-null   object 
 15  Data_selection       

In [26]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,5107,2025-05-18,2025-11-30
1,Dev_Train,4361,2024-10-01,2025-06-01


In [27]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3821631,4966e1c5-3510-4502-a753-f527b57f9210,8e264ec2-6363-48c3-ba7b-f6352761fdac,0.816216,2026-01-28T10:50:24.407553,2026-01-28T10:50:24.407553,Beta-Cash-Stack-Model,v1.1,"{""demo_score"": 0.5897851074761753, ""apps_score...",Cash December APPs Models Upgrade,69d10488-67b0-443e-a478-918a71a58819,2026-01-28T10:50:24.407553,{},Trench 1,android,Dev_Test,2025-11-20
1,3736233,8a9d856c-bb7f-479a-9645-3f4d6ff8d885,a921645c-023c-4b21-a329-28418a74830d,0.665282,2026-01-28T10:50:24.407553,2026-01-28T10:50:24.407553,Beta-Cash-Stack-Model,v1.1,"{""demo_score"": 0.6094913033674153, ""apps_score...",Cash December APPs Models Upgrade,c91d735d-6ce2-462b-b7cd-d39cfb3ca394,2026-01-28T10:50:24.407553,{},Trench 1,android,Dev_Test,2025-10-11
2,3781609,27421358-63b3-4824-baef-fad04edb18d9,d2539277-bb36-47c7-83ae-4bc62c49ed13,0.662455,2026-01-28T10:50:24.407553,2026-01-28T10:50:24.407553,Beta-Cash-Stack-Model,v1.1,"{""demo_score"": 0.609281808228018, ""apps_score""...",Cash December APPs Models Upgrade,66765df1-5a09-4b49-acdf-a9740ece3934,2026-01-28T10:50:24.407553,{},Trench 1,android,Dev_Test,2025-10-31
3,3635948,1057b8e7-12b2-4d70-a767-70fd10e1ebd6,065033d0-5c5e-4b06-a0df-3a77ac4722c2,0.476258,2026-01-28T10:50:24.408548,2026-01-28T10:50:24.408548,Beta-Cash-Stack-Model,v1.1,"{""demo_score"": 0.43907615146036677, ""apps_scor...",Cash December APPs Models Upgrade,a5b5b2e5-ae2c-443f-ad44-ddc864487156,2026-01-28T10:50:24.408548,{},Trench 1,android,Dev_Test,2025-08-25
4,3840410,2b99a460-c342-4f37-9230-0bd92a9c146d,2e272a45-a94c-4a2b-9e1c-48274c246b37,0.429238,2026-01-28T10:50:24.408548,2026-01-28T10:50:24.408548,Beta-Cash-Stack-Model,v1.1,"{""demo_score"": 0.4348320869306623, ""apps_score...",Cash December APPs Models Upgrade,f85fe605-0a66-4aea-90df-f8c0815a302a,2026-01-28T10:50:24.408548,{},Trench 1,android,Dev_Test,2025-11-27


In [28]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=915b419e-4748-4ad5-8373-3a2785ce4e2c>

## Trench 2

In [29]:
sq = """ 
WITH
  base AS (
    SELECT
      r.customer_id,
      r.digitalLoanAccountId,
      r.ln_appln_submit_datetime,
      r.trench_category,
      r.demo_score,
      r.apps_score,
      r.credo_score,
      r.stack_score,
      coalesce(
        CASE
          WHEN lower(r.ln_os_type) LIKE '%andro%' THEN 'android'
          WHEN lower(r.ln_os_type) LIKE '%os%' THEN 'ios'
          WHEN r.ln_os_type IS NULL THEN 'ios'
          ELSE 'ios'
          END,
        CASE
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%andro%'
            THEN 'android'
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%os%'
            THEN 'ios'
          WHEN lower(lmt.deviceType) LIKE '%andro%' THEN 'android'
          ELSE 'ios'
          END)
        osType,
      date(
        IF(
          lmt.new_loan_type = 'Flex-up',
          lmt.startApplyDateTime,
          lmt.termsAndConditionsSubmitDateTime))
        application_date,
      CASE
        WHEN dev_split LIKE 'Dev_Train' THEN 'Dev_Train'
        WHEN dev_split LIKE 'Dev_Test' THEN 'Dev_Test'
        ELSE dev_split
        END Data_selection
    FROM
      `worktable_data_analysis.cash_beta_trench2_applied_loans_backscored_20241001_20251130`
        r
    LEFT JOIN `risk_credit_mis.loan_master_table` lmt
      ON lmt.digitalLoanAccountId = r.digitalLoanAccountId
    WHERE lmt.disbursementDateTime IS NOT NULL
  )
SELECT 
customer_id,
digitalLoanAccountId,
ln_appln_submit_datetime,
trench_category,
demo_score,
apps_score,
credo_score,
stack_score,
osType,
application_date,
Data_selection,
FROM base
WHERE Data_selection IS NOT NULL
and osType = 'android';
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 13ce5dfb-f7e5-4446-9e82-06818dc6ed29 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (7770, 11)


In [30]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'Beta_cash_stack_score']

dfd = transform_data_v1_1(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash December APPs Models Upgrade') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7770 entries, 0 to 7769
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   customerId            7770 non-null   int64 
 1   digitalLoanAccountId  7770 non-null   object
 2   crifApplicationId     7770 non-null   object
 3   prediction            7770 non-null   int64 
 4   start_time            7770 non-null   object
 5   end_time              7770 non-null   object
 6   modelDisplayName      7770 non-null   object
 7   modelVersionId        7770 non-null   object
 8   calcFeature           7770 non-null   object
 9   subscription_name     7770 non-null   object
 10  message_id            7770 non-null   object
 11  publish_time          7770 non-null   object
 12  attributes            7770 non-null   object
 13  trenchCategory        7770 non-null   object
 14  deviceOs              7770 non-null   object
 15  Data_selection        7770 non-null   

In [31]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
0,3711927,95859702-37f4-4cdf-8c1f-34078a9264ff,876f20c3-34ea-4d2c-92f4-c33fedbbaac1,0,2026-01-28T10:59:04.797506,2026-01-28T10:59:04.797506,Beta-Cash-Stack-Model,v1.1,"{""demo_score"": 0.5927478804700357, ""apps_score...",Cash December APPs Models Upgrade,1162e96b-83ef-46b6-99fc-feb3fa909081,2026-01-28T10:59:04.797506,{},Trench 2,android,Dev_Test,2025-11-24
1,3540048,10d9752c-2c25-4811-adbd-ffcc908bd8dd,4a874826-9b80-40bb-b1f3-5692406b56ac,0,2026-01-28T10:59:04.798040,2026-01-28T10:59:04.798040,Beta-Cash-Stack-Model,v1.1,"{""demo_score"": 0.6064092106030952, ""apps_score...",Cash December APPs Models Upgrade,cbbcbb2b-8ee3-433c-a2d1-d3a3afdd25ca,2026-01-28T10:59:04.798040,{},Trench 2,android,Dev_Test,2025-11-04
2,3579598,3abf4585-0155-47c1-a080-6a35824d8d17,2350d453-8b95-4f83-87eb-e36ab9b01b9e,0,2026-01-28T10:59:04.798040,2026-01-28T10:59:04.798040,Beta-Cash-Stack-Model,v1.1,"{""demo_score"": 0.31752142144516565, ""apps_scor...",Cash December APPs Models Upgrade,96f7754f-9611-45a0-a329-a1f7e77eebca,2026-01-28T10:59:04.798040,{},Trench 2,android,Dev_Test,2025-10-30
3,3402428,a429f6e5-2b4a-4b3e-b484-5fe62c3a9173,056e1f68-2999-49c8-b1e6-7caae5687c24,0,2026-01-28T10:59:04.798040,2026-01-28T10:59:04.798040,Beta-Cash-Stack-Model,v1.1,"{""demo_score"": 0.4943371969096068, ""apps_score...",Cash December APPs Models Upgrade,421c98d0-1f6f-4c3a-8c56-c818d9b09ea7,2026-01-28T10:59:04.798040,{},Trench 2,android,Dev_Test,2025-11-15
4,2378078,3cb9c06f-eacb-4e9a-adf9-303f5908430e,fdedafa0-5d71-4038-b869-e71011639d18,0,2026-01-28T10:59:04.798561,2026-01-28T10:59:04.798561,Beta-Cash-Stack-Model,v1.1,"{""demo_score"": 0.5767201628816248, ""apps_score...",Cash December APPs Models Upgrade,45fafc13-1ae5-4eca-8321-1a4e51d46931,2026-01-28T10:59:04.798561,{},Trench 2,android,Dev_Test,2025-11-16


In [32]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,4121,2025-05-24,2025-11-30
1,Dev_Train,3649,2024-10-01,2025-06-01


In [33]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=feb63285-41b6-4e3c-a9a1-1b73f1bf61aa>

# Alpha Cash Stack Model

## Trench 1

In [34]:
sq = """ 
WITH
  base AS (
    SELECT
      r.customer_id,
      r.digitalLoanAccountId,
      r.ln_appln_submit_datetime,
      r.trench_category,
      r.demo_score,
      r.apps_score,
      r.credo_score,
      r.stack_score,
      r.cic_score,
      coalesce(
        CASE
          WHEN lower(r.ln_os_type) LIKE '%andro%' THEN 'android'
          WHEN lower(r.ln_os_type) LIKE '%os%' THEN 'ios'
          WHEN r.ln_os_type IS NULL THEN 'ios'
          ELSE 'ios'
          END,
        CASE
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%andro%'
            THEN 'android'
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%os%'
            THEN 'ios'
          WHEN lower(lmt.deviceType) LIKE '%andro%' THEN 'android'
          ELSE 'ios'
          END)
        osType,
      date(
        IF(
          lmt.new_loan_type = 'Flex-up',
          lmt.startApplyDateTime,
          lmt.termsAndConditionsSubmitDateTime))
        application_date,
      CASE
        WHEN dev_split LIKE 'Dev_Train' THEN 'Dev_Train'
        WHEN dev_split LIKE 'Dev_Test' THEN 'Dev_Test'
        ELSE dev_split
        END Data_selection
    FROM
      `worktable_data_analysis.cash_alpha_trench1_applied_loans_backscored_20241001_20251130`
        r
    LEFT JOIN `risk_credit_mis.loan_master_table` lmt
      ON lmt.digitalLoanAccountId = r.digitalLoanAccountId
    WHERE lmt.disbursementDateTime IS NOT NULL
  )
SELECT 
customer_id,
digitalLoanAccountId,
ln_appln_submit_datetime,
trench_category,
demo_score,
apps_score,
credo_score,
stack_score,
cic_score,
osType,
application_date,
Data_selection,
FROM base
WHERE Data_selection IS NOT NULL
and osType = 'android';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 1f03a93d-6016-499b-b1e4-c4d57c963408 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (9091, 12)


In [35]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score']

dfd = transform_data_v1_1(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash December APPs Models Upgrade') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            9091 non-null   int64  
 1   digitalLoanAccountId  9091 non-null   object 
 2   crifApplicationId     9091 non-null   object 
 3   prediction            9091 non-null   float64
 4   start_time            9091 non-null   object 
 5   end_time              9091 non-null   object 
 6   modelDisplayName      9091 non-null   object 
 7   modelVersionId        9091 non-null   object 
 8   calcFeature           9091 non-null   object 
 9   subscription_name     9091 non-null   object 
 10  message_id            9091 non-null   object 
 11  publish_time          9091 non-null   object 
 12  attributes            9091 non-null   object 
 13  trenchCategory        9091 non-null   object 
 14  deviceOs              9091 non-null   object 
 15  Data_selection       

In [37]:
dfd.sample(5)

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
3391,3600327,7e8b3f26-34fb-44ba-837f-4f7d6b202440,bb1dce15-5c7b-4bc9-9d7f-28fd679e651f,0.544612,2026-01-28T12:04:06.889320,2026-01-28T12:04:06.889320,Alpha-Cash-Stack-Model,v1.1,"{""demo_score"": 0.47669416506344026, ""apps_scor...",Cash December APPs Models Upgrade,ac95d7c6-551d-4532-a5de-4fa445abb7af,2026-01-28T12:04:06.889320,{},Trench 1,android,Dev_Test,2025-08-06
1491,3626971,39be54e5-8593-4d9c-ae19-e77eeb761e8e,70649cdd-8db3-47c3-9712-e626f4572e52,0.631914,2026-01-28T12:04:06.672235,2026-01-28T12:04:06.672235,Alpha-Cash-Stack-Model,v1.1,"{""demo_score"": 0.5864392342139578, ""apps_score...",Cash December APPs Models Upgrade,0855ad76-991a-4f61-bdb8-60f66479d140,2026-01-28T12:04:06.672235,{},Trench 1,android,Dev_Test,2025-08-17
5124,2896553,bdd0811c-d8df-4b09-a32c-22a25a93ebbd,46792b63-6a11-443e-a3f9-6c86cec4c814,0.261724,2026-01-28T12:04:07.082980,2026-01-28T12:04:07.082980,Alpha-Cash-Stack-Model,v1.1,"{""demo_score"": 0.3724048115450959, ""apps_score...",Cash December APPs Models Upgrade,54ba3f55-b4f5-40fa-b3c7-874d0821096f,2026-01-28T12:04:07.082980,{},Trench 1,android,Dev_Train,2024-10-13
1116,3726506,4b584e79-2246-4e7f-a0f8-54e11d4e36b3,3fd1ac8b-14f1-47ae-a16e-16addf00a0c8,0.672609,2026-01-28T12:04:06.631671,2026-01-28T12:04:06.631671,Alpha-Cash-Stack-Model,v1.1,"{""demo_score"": 0.48711586307006166, ""apps_scor...",Cash December APPs Models Upgrade,60c603f1-3509-46de-b9f7-45949e657de3,2026-01-28T12:04:06.631671,{},Trench 1,android,Dev_Test,2025-10-06
7512,3060393,6da03a0e-df3c-446b-ab4b-a6102267191e,a9124bc2-d4a8-4b4e-a825-5ef62a7fedb7,0.950408,2026-01-28T12:04:07.376072,2026-01-28T12:04:07.376072,Alpha-Cash-Stack-Model,v1.1,"{""demo_score"": 0.5464524234644476, ""apps_score...",Cash December APPs Models Upgrade,c81532fe-5e2e-41ac-a89b-b01494bcd68a,2026-01-28T12:04:07.376072,{},Trench 1,android,Dev_Train,2024-12-04


In [38]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,4865,2025-05-18,2025-11-30
1,Dev_Train,4226,2024-10-01,2025-06-01


In [39]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=49062729-468e-406f-b53f-d50d9dff49f2>

## Trench 2

In [40]:
sq = """ 
WITH
  base AS (
    SELECT
      r.customer_id,
      r.digitalLoanAccountId,
      r.ln_appln_submit_datetime,
      r.trench_category,
      r.demo_score,
      r.apps_score,
      r.credo_score,
      r.stack_score,
      r.cic_score,
      coalesce(
        CASE
          WHEN lower(r.ln_os_type) LIKE '%andro%' THEN 'android'
          WHEN lower(r.ln_os_type) LIKE '%os%' THEN 'ios'
          WHEN r.ln_os_type IS NULL THEN 'ios'
          ELSE 'ios'
          END,
        CASE
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%andro%'
            THEN 'android'
          WHEN lower(coalesce(lmt.osversion_v2, lmt.osVersion)) LIKE '%os%'
            THEN 'ios'
          WHEN lower(lmt.deviceType) LIKE '%andro%' THEN 'android'
          ELSE 'ios'
          END)
        osType,
      date(
        IF(
          lmt.new_loan_type = 'Flex-up',
          lmt.startApplyDateTime,
          lmt.termsAndConditionsSubmitDateTime))
        application_date,
      CASE
        WHEN dev_split LIKE 'Dev_Train' THEN 'Dev_Train'
        WHEN dev_split LIKE 'Dev_Test' THEN 'Dev_Test'
        ELSE dev_split
        END Data_selection
    FROM
      `worktable_data_analysis.cash_alpha_trench1_applied_loans_backscored_20241001_20251130`
        r
    LEFT JOIN `risk_credit_mis.loan_master_table` lmt
      ON lmt.digitalLoanAccountId = r.digitalLoanAccountId
    WHERE lmt.disbursementDateTime IS NOT NULL
  )
SELECT 
customer_id,
digitalLoanAccountId,
ln_appln_submit_datetime,
trench_category,
demo_score,
apps_score,
credo_score,
stack_score,
cic_score,
osType,
application_date,
Data_selection,
FROM base
WHERE Data_selection IS NOT NULL
and osType = 'android';
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID cb8de55a-c5b7-48f2-a661-fb6a16532f0f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (9091, 12)


In [41]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score']

dfd = transform_data_v1_1(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash December APPs Models Upgrade') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            9091 non-null   int64  
 1   digitalLoanAccountId  9091 non-null   object 
 2   crifApplicationId     9091 non-null   object 
 3   prediction            9091 non-null   float64
 4   start_time            9091 non-null   object 
 5   end_time              9091 non-null   object 
 6   modelDisplayName      9091 non-null   object 
 7   modelVersionId        9091 non-null   object 
 8   calcFeature           9091 non-null   object 
 9   subscription_name     9091 non-null   object 
 10  message_id            9091 non-null   object 
 11  publish_time          9091 non-null   object 
 12  attributes            9091 non-null   object 
 13  trenchCategory        9091 non-null   object 
 14  deviceOs              9091 non-null   object 
 15  Data_selection       

In [43]:
dfd.sample(5)

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs,Data_selection,Application_date
8474,3048694,340e4f04-2134-4370-aff4-fd2f29fd8b0c,d7af941a-b8f2-40be-8e6e-a03440e70ec0,0.808652,2026-01-28T12:32:45.827529,2026-01-28T12:32:45.827529,Alpha-Cash-Stack-Model,v1.1,"{""demo_score"": 0.5767184009249257, ""apps_score...",Cash December APPs Models Upgrade,753a3f2c-5c42-40fd-b7a9-888af800ac69,2026-01-28T12:32:45.827529,{},Trench 2,android,Dev_Train,2024-11-23
8218,3061635,d14296a7-04f6-4125-8b2a-c215a8b5c53b,edac32eb-046d-43d1-9c1a-49be850b3e33,0.628730,2026-01-28T12:32:45.795158,2026-01-28T12:32:45.795158,Alpha-Cash-Stack-Model,v1.1,"{""demo_score"": 0.5512367876750216, ""apps_score...",Cash December APPs Models Upgrade,f269c3f0-ec85-40f9-a4d5-da02e1046ff5,2026-01-28T12:32:45.795158,{},Trench 2,android,Dev_Train,2024-11-28
8292,3358889,c977c3d2-d2d7-4529-882e-89f1b02a6f12,4d49b779-4e1a-4636-9b60-e40763c286e5,0.401993,2026-01-28T12:32:45.802137,2026-01-28T12:32:45.802137,Alpha-Cash-Stack-Model,v1.1,"{""demo_score"": 0.373829611348188, ""apps_score""...",Cash December APPs Models Upgrade,6fbc21b6-983e-42b8-a054-c630bbf6b24f,2026-01-28T12:32:45.802137,{},Trench 2,android,Dev_Train,2025-04-04
8227,2941801,1379933e-12bb-4fcb-b626-91cb5041d40c,60cd3b08-6838-43b0-a23c-70f71ba0f461,0.934985,2026-01-28T12:32:45.795158,2026-01-28T12:32:45.795158,Alpha-Cash-Stack-Model,v1.1,"{""demo_score"": 0.6989944535688777, ""apps_score...",Cash December APPs Models Upgrade,3f7d8364-c77b-49d8-8a5e-be0176a9e08f,2026-01-28T12:32:45.795158,{},Trench 2,android,Dev_Train,2024-10-16
7967,2899143,8e9e849a-3d21-4220-a60d-97f11a8fa346,aa4a9e21-3353-476d-ba8a-34649c1eef50,0.224389,2026-01-28T12:32:45.744670,2026-01-28T12:32:45.744670,Alpha-Cash-Stack-Model,v1.1,"{""demo_score"": 0.41337939535058676, ""apps_scor...",Cash December APPs Models Upgrade,a6a88c65-f4e6-43f6-a535-3e41da2fcbbc,2026-01-28T12:32:45.744670,{},Trench 2,android,Dev_Train,2024-10-04


In [42]:
result = dfd.groupby('Data_selection').agg(
    digitalLoanAccountId_count=('digitalLoanAccountId', 'count'),
    Application_date_min=('Application_date', 'min'),
    Application_date_max=('Application_date', 'max')
).reset_index()

result

,Data_selection,digitalLoanAccountId_count,Application_date_min,Application_date_max
0,Dev_Test,4865,2025-05-18,2025-11-30
1,Dev_Train,4226,2024-10-01,2025-06-01


In [44]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=146f8d0b-14b8-4256-991c-1a6a2145ba34>